<a href="https://colab.research.google.com/github/mjbrody/lily58-wireless-view-zmk-config/blob/main/FengShuiAttempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# FengShui GIS Analysis - Colab-ready Python notebook
# Purpose: compute street orientation deviation (hierarchy-weighted) and % buildings that are
# facing water within 100 m and backed by higher elevation. Produces composite Feng Shui score.

# ---------- Cell 1: Install dependencies ----------
# Run this cell first in Colab
!pip install osmnx geopandas rasterio shapely pyproj scipy pandas fiona rtree requests elevation


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 37.1 MB/s eta 0:00:00


In [3]:
# ---------- Cell 2: Imports ----------
import os, math, tempfile
import numpy as np, pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point, LineString, Polygon
import rasterio
from rasterio import mask
import json


import logging
ox.settings.log_console = False
ox.settings.use_cache = True
logging.getLogger('osmnx').setLevel(logging.ERROR)

In [4]:
# ---------- Helper functions (Cells 3–6 combined, cleaned for Colab) ----------

import math
import numpy as np
import pandas as pd
from shapely.geometry import Point

# --- Compute azimuth/bearing between two points ---
def azimuth(p1, p2):
    """Return bearing in degrees where north=0, increasing clockwise."""
    x1, y1 = p1.x, p1.y
    x2, y2 = p2.x, p2.y
    angle = math.degrees(math.atan2((x2 - x1), (y2 - y1)))
    return (angle + 360) % 360


# --- Compute deviation from nearest cardinal direction (N, E, S, W) ---
def deviation_from_cardinal_series(bearings):
    cardinals = np.array([0, 90, 180, 270])

    def dev(b):
        diffs = np.abs(((b - cardinals + 180) % 360) - 180)
        return diffs.min()

    return bearings.apply(dev)


# --- Extract segment bearings and apply hierarchy weights ---
def segment_bearings_with_hierarchy(edges_gdf, default_weight_map=None):
    if default_weight_map is None:
        default_weight_map = {
            "motorway": 3.0,
            "trunk": 3.0,
            "primary": 2.5,
            "secondary": 2.0,
            "tertiary": 1.5,
            "unclassified": 1.0,
            "residential": 1.0,
            "service": 0.8,
        }

    rows = []
    for idx, row in edges_gdf.iterrows():
        geom = row.geometry
        if geom is None:
            continue
        hw = row.get("highway")
        if isinstance(hw, list) and len(hw) > 0:
            hw0 = hw[0]
        else:
            hw0 = hw
        w = default_weight_map.get(hw0, 1.0)

        # handle both LineString and MultiLineString
        lines = list(geom.geoms) if geom.geom_type == "MultiLineString" else [geom]

        for line in lines:
            coords = list(line.coords)
            for i in range(len(coords) - 1):
                p1 = Point(coords[i])
                p2 = Point(coords[i + 1])
                b = azimuth(p1, p2)
                seg_len = p1.distance(p2)
                rows.append({"bearing": b, "length": seg_len, "weight": w})

    segs = pd.DataFrame(rows)
    return segs


# --- DEM sampling helper ---
def sample_elevation_point(dem_path, point_geom):
    """Return elevation (float) for a given point from DEM raster."""
    import rasterio

    with rasterio.open(dem_path) as src:
        try:
            coords = [(point_geom.x, point_geom.y)]
            vals = list(src.sample(coords))
            v = vals[0][0]
            if v == src.nodata:
                return np.nan
            return float(v)
        except Exception:
            return np.nan


# --- Building principal orientation helper ---
def building_orientation(building_poly):
    """Estimate dominant orientation of a building polygon."""
    mrr = building_poly.minimum_rotated_rectangle
    coords = list(mrr.exterior.coords)
    max_len = 0
    ang = 0
    for i in range(len(coords) - 1):
        a = Point(coords[i])
        b = Point(coords[i + 1])
        l = a.distance(b)
        if l > max_len:
            max_len = l
            dx = coords[i + 1][0] - coords[i][0]
            dy = coords[i + 1][1] - coords[i][1]
            ang = (math.degrees(math.atan2(dx, dy)) + 360) % 360
    return ang

print("✅ Helper functions loaded successfully.")

✅ Helper functions loaded successfully.


In [5]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point

def analyze_city(
    place_name,
    dem_path=None,
    max_water_dist_m=100,
    ang_tolerance=45,
    project_crs="EPSG:3857"
):
    """
    Analyze Feng Shui characteristics of a city.

    Args:
        place_name (str): e.g., "Singapore" or "Charlotte, North Carolina, USA"
        dem_path (str): path to DEM .tif file (optional)
        max_water_dist_m (float): max distance from building to water in meters
        ang_tolerance (float): max angle difference between building and water direction
        project_crs (str): projected CRS for distance calculations
    """

    print(f"Fetching OSM data for {place_name} ...")

    # --- Fetch boundary polygon ---
    area_gdf = ox.geocode_to_gdf(place_name)
    poly = area_gdf.geometry.iloc[0]

    # --- Fetch streets (drive network) and buildings ---
    G = ox.graph_from_polygon(poly, network_type="drive")
    nodes, edges = ox.graph_to_gdfs(G)
    tags = {"building": True}
    buildings = ox.features_from_polygon(poly, tags) # Changed from geometries_from_polygon
    buildings = buildings[buildings.geometry.notnull()]
    buildings = buildings[buildings.geometry.type.isin(["Polygon", "MultiPolygon"])]

    # --- Reproject for distance-based calcs ---
    edges = edges.to_crs(project_crs)
    buildings = buildings.to_crs(project_crs)

    # --- Compute street segment bearings (hierarchy-weighted) ---
    segs = segment_bearings_with_hierarchy(edges)
    segs["deviation"] = deviation_from_cardinal_series(segs["bearing"])
    segs["weighted_len"] = segs["length"] * segs["weight"]

    # Mean deviation weighted by street length × hierarchy weight
    mean_dev = np.average(segs["deviation"], weights=segs["weighted_len"])
    orientation_score = max(0, 100 * (1 - (mean_dev / 45.0)))  # 0–100 scale

    # --- Water features from OSM ---
    water_tags = {"natural": "water", "waterway": "riverbank", "water": "lake"}
    water = ox.features_from_polygon(poly, tags=water_tags) # Changed from geometries_from_polygon
    water = water[water.geometry.notnull()]
    if len(water) == 0:
        print(f"⚠️ No water polygons found for {place_name}.")
    water = water.to_crs(project_crs)
    water_centroids = water.geometry.centroid.reset_index(drop=True)

    # --- Analyze each building ---
    results = []
    for idx, row in buildings.iterrows():
        geom = row.geometry
        rep = geom.representative_point()
        bearing_build = building_orientation(geom)

        # --- Nearest water centroid ---
        if len(water_centroids) > 0:
            dists = water_centroids.distance(rep)
            minidx = dists.idxmin()
            dist_to_water = dists.min()
            nearest = water_centroids[minidx]
            dx = nearest.x - rep.x
            dy = nearest.y - rep.y
            bearing_to_water = (np.degrees(np.arctan2(dx, dy)) + 360) % 360
        else:
            dist_to_water = np.nan
            bearing_to_water = np.nan

        faces = False
        if not np.isnan(dist_to_water) and dist_to_water <= max_water_dist_m:
            ang_diff = min(
                abs(((bearing_build - bearing_to_water + 180) % 360) - 180),
                abs(((bearing_build + 180 - bearing_to_water + 180) % 360) - 180),
            )
            if ang_diff <= ang_tolerance:
                faces = True

        # --- Check elevation backing if DEM provided ---
        backed = False
        if dem_path and not np.isnan(dist_to_water):
            ang_rad = np.radians(bearing_to_water)
            dx = np.sin(ang_rad) * 20
            dy = np.cos(ang_rad) * 20
            front_pt = Point(rep.x + dx, rep.y + dy)
            back_pt = Point(rep.x - dx, rep.y - dy)
            elev_front = sample_elevation_point(dem_path, front_pt)
            elev_back = sample_elevation_point(dem_path, back_pt)
            if not np.isnan(elev_front) and not np.isnan(elev_back):
                if elev_back > elev_front + 1.0:
                    backed = True

        results.append({
            "id": idx,
            "bearing": bearing_build,
            "dist_to_water_m": float(dist_to_water) if not np.isnan(dist_to_water) else np.nan,
            "bearing_to_water": float(bearing_to_water) if not np.isnan(bearing_to_water) else np.nan,
            "faces_water": faces,
            "backed_by_higher": backed
        })

    bdf = pd.DataFrame(results)

    # --- Compute scores ---
    if len(bdf) == 0:
        pct_water_backed = 0.0
    else:
        pct_water_backed = 100.0 * ((bdf["faces_water"]) & (bdf["backed_by_higher"])).sum() / len(bdf)

    water_score = pct_water_backed
    composite_score = 0.6 * orientation_score + 0.4 * water_score

    # --- Export GeoJSON of building metrics ---
    output_dir = "/content/fengshui_output"
    os.makedirs(output_dir, exist_ok=True)
    buildings = buildings.reset_index(drop=True)
    for col in ["bearing", "dist_to_water_m", "faces_water", "backed_by_higher"]:
        buildings[col] = bdf[col] if col in bdf else np.nan

    out_geo = os.path.join(
        output_dir,
        f"{place_name.replace(',', '').replace(' ', '_')}_buildings.geojson",
    )
    buildings.to_file(out_geo, driver="GeoJSON")

    stats = {
        "place": place_name,
        "mean_deviation_deg": mean_dev,
        "orientation_score": orientation_score,
        "pct_water_backed": pct_water_backed,
        "composite_score": composite_score,
        "output_geojson": out_geo,
    }

    print(f"✅ Analysis complete for {place_name}")
    print(f"Composite Feng Shui Score: {composite_score:.2f}")
    return stats

print("✅ analyze_city() function ready to use.")

✅ analyze_city() function ready to use.


In [7]:
stats_sg = analyze_city("Singapore", dem_path=None)
stats_clt = analyze_city("Charlotte, North Carolina, USA", dem_path=None)
print(stats_sg)
print(stats_clt)


Fetching OSM data for Singapore ...
✅ Analysis complete for Singapore
Composite Feng Shui Score: 31.11
Fetching OSM data for Charlotte, North Carolina, USA ...
✅ Analysis complete for Charlotte, North Carolina, USA
Composite Feng Shui Score: 30.39
{'place': 'Singapore', 'mean_deviation_deg': np.float64(21.67074960990466), 'orientation_score': np.float64(51.842778644656306), 'pct_water_backed': np.float64(0.0), 'composite_score': np.float64(31.10566718679378), 'output_geojson': '/content/fengshui_output/Singapore_buildings.geojson'}
{'place': 'Charlotte, North Carolina, USA', 'mean_deviation_deg': np.float64(22.205469961100174), 'orientation_score': np.float64(50.65451119755517), 'pct_water_backed': np.float64(0.0), 'composite_score': np.float64(30.392706718533102), 'output_geojson': '/content/fengshui_output/Charlotte_North_Carolina_USA_buildings.geojson'}


In [9]:
stats_tp = analyze_city("Taipei", dem_path=None)
stats_zrc = analyze_city("Zurich", dem_path=None)
print(stats_tp)
print(stats_zrc)

Fetching OSM data for Taipei ...
✅ Analysis complete for Taipei
Composite Feng Shui Score: 36.96
Fetching OSM data for Zurich ...
✅ Analysis complete for Zurich
Composite Feng Shui Score: 27.92
{'place': 'Taipei', 'mean_deviation_deg': np.float64(17.280279109017407), 'orientation_score': np.float64(61.5993797577391), 'pct_water_backed': np.float64(0.0), 'composite_score': np.float64(36.95962785464346), 'output_geojson': '/content/fengshui_output/Taipei_buildings.geojson'}
{'place': 'Zurich', 'mean_deviation_deg': np.float64(24.060556232506645), 'orientation_score': np.float64(46.53209726109634), 'pct_water_backed': np.float64(0.0), 'composite_score': np.float64(27.919258356657803), 'output_geojson': '/content/fengshui_output/Zurich_buildings.geojson'}


In [6]:
stats_mlp = analyze_city("Minneapolis, MN, USA", dem_path=None)
stats_van = analyze_city("Vancouver, BC, Canada", dem_path=None)
print(stats_mlp)
print(stats_van)

Fetching OSM data for Minneapolis, MN, USA ...
✅ Analysis complete for Minneapolis, MN, USA
Composite Feng Shui Score: 50.38
Fetching OSM data for Vancouver, BC, Canada ...
✅ Analysis complete for Vancouver, BC, Canada
Composite Feng Shui Score: 49.92
{'place': 'Minneapolis, MN, USA', 'mean_deviation_deg': np.float64(7.214156546806146), 'orientation_score': np.float64(83.96854100709746), 'pct_water_backed': np.float64(0.0), 'composite_score': np.float64(50.381124604258474), 'output_geojson': '/content/fengshui_output/Minneapolis_MN_USA_buildings.geojson'}
{'place': 'Vancouver, BC, Canada', 'mean_deviation_deg': np.float64(7.562683601113046), 'orientation_score': np.float64(83.19403644197101), 'pct_water_backed': np.float64(0.0), 'composite_score': np.float64(49.916421865182606), 'output_geojson': '/content/fengshui_output/Vancouver_BC_Canada_buildings.geojson'}


In [1]:
import geopandas as gpd
gdf = gpd.read_file(stats['/content/fengshui_output/Vancouver_BC_Canada_buildings.geojson'])
gdf.head()


NameError: name 'stats' is not defined

In [ ]:
### V2: no elevation

In [5]:
# ---------- Simplified analyze_city() (no elevation required, OSMnx ≥ 2.0) ----------

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import math

def analyze_city(
    place_name,
    max_water_dist_m=100,
    ang_tolerance=45,
    project_crs="EPSG:3857"
):
    """
    Analyze simplified Feng Shui characteristics of a city.

    Args:
        place_name (str): e.g., "Singapore" or "Charlotte, North Carolina, USA"
        max_water_dist_m (float): max distance from building to water in meters
        ang_tolerance (float): angle difference (deg) between building orientation and water direction
        project_crs (str): projected CRS for distance calculations
    """

    print(f"Fetching OSM data for {place_name} ...")

    # --- Fetch boundary polygon ---
    area_gdf = ox.geocode_to_gdf(place_name)
    poly = area_gdf.geometry.iloc[0]

    # --- Fetch streets (drive network) and buildings ---
    G = ox.graph_from_polygon(poly, network_type="drive")
    nodes, edges = ox.graph_to_gdfs(G)
    tags = {"building": True}
    buildings = ox.features_from_polygon(poly, tags)
    buildings = buildings[buildings.geometry.notnull()]
    buildings = buildings[buildings.geometry.type.isin(["Polygon", "MultiPolygon"])]

    # --- Reproject ---
    edges = edges.to_crs(project_crs)
    buildings = buildings.to_crs(project_crs)

    # --- Compute street bearings & hierarchy weights ---
    segs = segment_bearings_with_hierarchy(edges)
    segs["deviation"] = deviation_from_cardinal_series(segs["bearing"])
    segs["weighted_len"] = segs["length"] * segs["weight"]

    mean_dev = np.average(segs["deviation"], weights=segs["weighted_len"])
    orientation_score = max(0, 100 * (1 - (mean_dev / 45.0)))

    # --- Fetch water features ---
    water_tags = {"natural": "water", "waterway": "riverbank", "water": "lake"}
    water = ox.features_from_polygon(poly, tags=water_tags)
    water = water[water.geometry.notnull()]
    if len(water) == 0:
        print(f"⚠️ No water polygons found for {place_name}.")
    water = water.to_crs(project_crs)
    water_centroids = water.geometry.centroid.reset_index(drop=True)

    # --- Evaluate each building’s orientation vs. water ---
    results = []
    for idx, row in buildings.iterrows():
        geom = row.geometry
        rep = geom.representative_point()
        bearing_build = building_orientation(geom)

        # Nearest water body
        if len(water_centroids) > 0:
            dists = water_centroids.distance(rep)
            minidx = dists.idxmin()
            dist_to_water = dists.min()
            nearest = water_centroids[minidx]
            dx = nearest.x - rep.x
            dy = nearest.y - rep.y
            bearing_to_water = (np.degrees(np.arctan2(dx, dy)) + 360) % 360
        else:
            dist_to_water = np.nan
            bearing_to_water = np.nan

        faces = False
        if not np.isnan(dist_to_water) and dist_to_water <= max_water_dist_m:
            ang_diff = min(
                abs(((bearing_build - bearing_to_water + 180) % 360) - 180),
                abs(((bearing_build + 180 - bearing_to_water + 180) % 360) - 180),
            )
            if ang_diff <= ang_tolerance:
                faces = True

        results.append({
            "id": idx,
            "bearing": bearing_build,
            "dist_to_water_m": float(dist_to_water) if not np.isnan(dist_to_water) else np.nan,
            "bearing_to_water": float(bearing_to_water) if not np.isnan(bearing_to_water) else np.nan,
            "faces_water": faces
        })

    bdf = pd.DataFrame(results)
    pct_faces_water = 100.0 * (bdf["faces_water"].sum() / len(bdf)) if len(bdf) > 0 else 0.0

    water_score = pct_faces_water
    composite_score = 0.6 * orientation_score + 0.4 * water_score

    # --- Export results ---
    output_dir = "/content/fengshui_output"
    os.makedirs(output_dir, exist_ok=True)
    buildings = buildings.reset_index(drop=True)
    for col in ["bearing", "dist_to_water_m", "faces_water"]:
        buildings[col] = bdf[col] if col in bdf else np.nan

    out_geo = os.path.join(
        output_dir,
        f"{place_name.replace(',', '').replace(' ', '_')}_buildings.geojson"
    )
    buildings.to_file(out_geo, driver="GeoJSON")

    stats = {
        "place": place_name,
        "mean_deviation_deg": mean_dev,
        "orientation_score": orientation_score,
        "pct_faces_water": pct_faces_water,
        "composite_score": composite_score,
        "output_geojson": out_geo,
    }

    print(f"✅ Analysis complete for {place_name}")
    print(f"Composite Feng Shui Score: {composite_score:.2f}")
    return stats

print("✅ Simplified analyze_city() ready.")


✅ Simplified analyze_city() ready.


In [8]:
# ---------- Helper functions (complete, clean, Colab-ready) ----------

import math
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString

def azimuth(p1, p2):
    """Return bearing in degrees where north=0 increasing clockwise."""
    x1, y1 = p1.x, p1.y
    x2, y2 = p2.x, p2.y
    angle = math.degrees(math.atan2((x2 - x1), (y2 - y1)))
    return (angle + 360) % 360


def segment_bearings_with_hierarchy(edges):
    """
    Compute bearing and assign a hierarchy weight for each street segment.
    Primary roads contribute more to the orientation score.
    """
    rows = []
    for idx, row in edges.iterrows():
        geom = row.geometry
        if geom is None or geom.is_empty:
            continue

        if geom.geom_type == "MultiLineString":
            lines = list(geom.geoms)
        else:
            lines = [geom]

        for line in lines:
            if not isinstance(line, LineString):
                continue
            coords = list(line.coords)
            if len(coords) < 2:
                continue
            p1 = gpd.points_from_xy([coords[0][0]], [coords[0][1]])[0]
            p2 = gpd.points_from_xy([coords[-1][0]], [coords[-1][1]])[0]
            bearing = azimuth(p1, p2)
            length = line.length

            # Assign weight by highway type if available
            highway = row.get("highway", "")
            if isinstance(highway, list):
                highway = highway[0]

            if highway in ["motorway", "trunk", "primary"]:
                weight = 3
            elif highway in ["secondary", "tertiary"]:
                weight = 2
            else:
                weight = 1

            rows.append({
                "bearing": bearing,
                "length": length,
                "weight": weight
            })

    return gpd.GeoDataFrame(rows)


def deviation_from_cardinal_series(series):
    """Return angular deviation from nearest cardinal direction (0°, 90°, 180°, 270°)."""
    return series.apply(lambda b: min(abs(((b - 0) % 180)), abs(((b - 90) % 180))))


def building_orientation(poly):
    """Compute dominant building orientation in degrees."""
    if poly is None or poly.is_empty:
        return np.nan
    min_rect = poly.minimum_rotated_rectangle
    coords = list(min_rect.exterior.coords)
    if len(coords) < 2:
        return np.nan
    p1, p2 = gpd.points_from_xy([coords[0][0]], [coords[0][1]])[0], gpd.points_from_xy([coords[1][0]], [coords[1][1]])[0]
    return azimuth(p1, p2)


In [10]:
stats_sg = analyze_city("Singapore")
stats_clt = analyze_city("Charlotte, North Carolina, USA")

print(stats_sg)
print(stats_clt)


Fetching OSM data for Singapore ...
✅ Analysis complete for Singapore
Composite Feng Shui Score: 0.86
Fetching OSM data for Charlotte, North Carolina, USA ...
✅ Analysis complete for Charlotte, North Carolina, USA
Composite Feng Shui Score: 1.55
{'place': 'Singapore', 'mean_deviation_deg': np.float64(44.80595895327156), 'orientation_score': np.float64(0.43120232606319586), 'pct_faces_water': np.float64(1.5128387504095775), 'composite_score': np.float64(0.8638568958017485), 'output_geojson': '/content/fengshui_output/Singapore_buildings.geojson'}
{'place': 'Charlotte, North Carolina, USA', 'mean_deviation_deg': np.float64(44.04795512555585), 'orientation_score': np.float64(2.1156552765425496), 'pct_faces_water': np.float64(0.7073793478715834), 'composite_score': np.float64(1.5523449050741631), 'output_geojson': '/content/fengshui_output/Charlotte_North_Carolina_USA_buildings.geojson'}


In [11]:
m_sg = visualize_fengshui("Singapore", stats_sg["output_geojson"])
m_sg

NameError: name 'visualize_fengshui' is not defined

In [14]:
# ---------- Visualization cell: Feng Shui map viewer ----------

import folium
import geopandas as gpd
import osmnx as ox
from folium import features

def visualize_fengshui(place_name, geojson_path):
    """
    Create an interactive Folium map visualizing Feng Shui features.
    Args:
        place_name (str): e.g., "Singapore"
        geojson_path (str): path to GeoJSON output from analyze_city()
    """
    # Load building layer
    buildings = gpd.read_file(geojson_path)
    centroid = buildings.unary_union.centroid
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12, tiles="cartodb positron")

    # Water features
    water_tags = {"natural": "water", "waterway": "riverbank", "water": "lake"}
    water = ox.features_from_place(place_name, tags=water_tags)
    water = water[water.geometry.notnull()].to_crs(buildings.crs)
    folium.GeoJson(
        water,
        name="Water",
        style_function=lambda x: {"color": "#6bb6ff", "weight": 1, "fillColor": "#a3d3ff", "fillOpacity": 0.5},
    ).add_to(m)

    # Buildings facing water (blue) vs not (gray)
    facing = buildings[buildings["faces_water"] == True]
    not_facing = buildings[buildings["faces_water"] == False]

    folium.GeoJson(
        not_facing,
        name="Buildings (not facing water)",
        style_function=lambda x: {"color": "#999999", "weight": 0.3, "fillOpacity": 0.3},
    ).add_to(m)

    folium.GeoJson(
        facing,
        name="Buildings (facing water)",
        style_function=lambda x: {"color": "#0047ab", "weight": 0.5, "fillOpacity": 0.7},
        highlight_function=lambda x: {"weight": 2, "color": "#003366"},
        tooltip=folium.GeoJsonTooltip(fields=["dist_to_water_m"]),
    ).add_to(m)

    # Streets for context
    G = ox.graph_from_place(place_name, network_type="drive")
    edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
    edges = edges.to_crs(buildings.crs)
    folium.GeoJson(
        edges,
        name="Major Streets",
        style_function=lambda x: {"color": "#333333", "weight": 0.4, "opacity": 0.5},
    ).add_to(m)

    folium.LayerControl().add_to(m)
    print(f"✅ Map ready for {place_name}")
    return m

# Example usage:
m_sg = visualize_fengshui("Singapore", stats_sg["output_geojson"])
m_sg


KeyboardInterrupt: 

In [13]:
m_sg = visualize_fengshui("Singapore", stats_sg["output_geojson"])
m_sg


NameError: name 'visualize_fengshui' is not defined

In [15]:
stats_van = analyze_city("Vancouver, BC, Canada")
stats_zrc = analyze_city("Zurich")
print(stats_van)
print(stats_zrc)

Fetching OSM data for Vancouver, BC, Canada ...
✅ Analysis complete for Vancouver, BC, Canada
Composite Feng Shui Score: 35.21
Fetching OSM data for Zurich ...
✅ Analysis complete for Zurich
Composite Feng Shui Score: 1.25
{'place': 'Vancouver, BC, Canada', 'mean_deviation_deg': np.float64(18.62342665596486), 'orientation_score': np.float64(58.6146074311892), 'pct_faces_water': np.float64(0.09960231775320436), 'composite_score': np.float64(35.2086053858148), 'output_geojson': '/content/fengshui_output/Vancouver_BC_Canada_buildings.geojson'}
{'place': 'Zurich', 'mean_deviation_deg': np.float64(44.464933738750794), 'orientation_score': np.float64(1.1890361361093449), 'pct_faces_water': np.float64(1.3460157932519743), 'composite_score': np.float64(1.2518279989663967), 'output_geojson': '/content/fengshui_output/Zurich_buildings.geojson'}


In [16]:
stats_hk = analyze_city("Hong Kong")
stats_vie = analyze_city("Vienna")
print(stats_hk)
print(stats_vie)

Fetching OSM data for Hong Kong ...
✅ Analysis complete for Hong Kong
Composite Feng Shui Score: 1.43
Fetching OSM data for Vienna ...
✅ Analysis complete for Vienna
Composite Feng Shui Score: 9.16
{'place': 'Hong Kong', 'mean_deviation_deg': np.float64(45.29923054861219), 'orientation_score': 0, 'pct_faces_water': np.float64(3.5724774128906316), 'composite_score': np.float64(1.4289909651562527), 'output_geojson': '/content/fengshui_output/Hong_Kong_buildings.geojson'}
{'place': 'Vienna', 'mean_deviation_deg': np.float64(38.37735171094473), 'orientation_score': np.float64(14.7169961979006), 'pct_faces_water': np.float64(0.8168028004667445), 'composite_score': np.float64(9.156918838927057), 'output_geojson': '/content/fengshui_output/Vienna_buildings.geojson'}
